# Fish-Speech-1.5による日本語テキスト→音声変換アプリ（修正版）

## 準備
以下のセルを実行して、必要なライブラリをインストールします。初回のみ実行してください。

In [ ]:
# 必要なライブラリのインストール（最初に一度だけ実行）
!pip install torch torchaudio numpy scipy ipywidgets
!pip install gradio>=3.32.0
!pip install soundfile
!pip install pypinyin==0.48.0 loguru

## モデルの直接ダウンロード
Fish-Speech-1.5モデルをHugging Faceから直接ダウンロードします

In [ ]:
import os
import huggingface_hub

# モデルをダウンロードするディレクトリを作成
os.makedirs("models", exist_ok=True)

# モデルファイルのダウンロード
model_path = huggingface_hub.hf_hub_download(
    repo_id="fishaudio/fish-speech-1.5", 
    filename="fish-speech-1.0-epoch0100.onnx",
    cache_dir="./models"
)

print(f"モデルをダウンロードしました: {model_path}")

## アプリの実装（ONNXモデル使用版）
Fish-Speech-1.5のONNXモデルを使用した実装

In [ ]:
import numpy as np
import torch
import torchaudio
import onnxruntime as ort
import os
import io
import tempfile
from IPython.display import Audio, display, HTML
import ipywidgets as widgets
import scipy.io.wavfile as wavfile
from transformers import AutoTokenizer
import time

# ONNX Runtime セッションの初期化
def initialize_model(model_path):
    print(f"モデルを初期化しています: {model_path}")
    
    # GPUが利用可能かどうかをチェック
    if torch.cuda.is_available():
        print("CUDA GPUが利用可能です")
        providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
    else:
        print("CPUを使用します")
        providers = ['CPUExecutionProvider']
    
    # ONNXセッションの作成
    try:
        session = ort.InferenceSession(model_path, providers=providers)
        print("モデルの初期化が完了しました")
        return session
    except Exception as e:
        print(f"モデルの初期化に失敗しました: {e}")
        return None

# テキストの前処理
def preprocess_text(text, language="ja"):
    # 簡易的な前処理（実際には言語に応じた前処理が必要）
    text = text.strip()
    
    # 日本語文字が含まれているか確認
    has_japanese = any([ord(char) > 0x3000 for char in text])
    detected_language = "ja" if has_japanese else "en"
    
    if language != detected_language:
        print(f"警告: 言語指定({language})と検出された言語({detected_language})が異なります")
    
    return text

# テキストを音声に変換する関数（簡易版）
def simple_text_to_speech(text, language="ja", speed=1.0):
    """
    簡易版のテキスト→音声変換関数（フォールバック用）
    """
    # サンプリングレート
    sample_rate = 24000
    
    # 一時ファイルを作成
    temp_dir = tempfile.gettempdir()
    output_path = os.path.join(temp_dir, 'output_simple.wav')
    
    # テキストの長さに応じたシンプルな音声を生成（デモ用）
    duration = min(len(text) * 0.1, 10)  # テキストの長さに比例した音声の長さ（最大10秒）
    
    # 時間配列の作成
    t = np.linspace(0, duration, int(duration * sample_rate), endpoint=False)
    
    # 基本周波数（テキストの種類によって変化させるデモ）
    if language == "ja":
        # 日本語テキストの場合
        freq = 440  # A4の周波数
    else:
        # 英語など他の言語の場合
        freq = 392  # G4の周波数
    
    # シンプルな音声信号の生成
    waveform = 0.5 * np.sin(2 * np.pi * freq * t) * np.exp(-t/duration)
    
    # 速度調整（リサンプリング）
    if speed != 1.0:
        # リサンプリングの比率
        resample_ratio = 1.0 / speed
        new_length = int(len(waveform) * resample_ratio)
        
        # リサンプリング
        waveform_resampled = np.interp(
            np.linspace(0, len(waveform) - 1, new_length),
            np.arange(len(waveform)),
            waveform
        )
        
        waveform = waveform_resampled
    
    # WAVファイルとして保存
    wavfile.write(output_path, sample_rate, waveform.astype(np.float32))
    
    return output_path

# ファイルからテキストを読み込む関数
def read_text_file(uploaded_file):
    """
    アップロードされたファイルからテキストを読み込む
    
    Parameters:
    -----------
    uploaded_file : UploadedFile
        アップロードされたテキストファイル
        
    Returns:
    --------
    str
        ファイルの内容
    """
    content = uploaded_file.read()
    # エンコーディングを自動検出（日本語ファイルの場合、utf-8やshift-jisなど様々な可能性がある）
    encodings = ['utf-8', 'shift-jis', 'euc-jp', 'iso-2022-jp']
    
    for encoding in encodings:
        try:
            text = content.decode(encoding)
            return text
        except UnicodeDecodeError:
            continue
    
    # どのエンコーディングでも失敗した場合
    raise ValueError("ファイルのエンコーディングを検出できませんでした。")

# ダウンロードリンクを作成する関数
def create_download_link(audio_data, filename, text):
    """
    音声データをダウンロードするためのHTMLリンクを作成
    
    Parameters:
    -----------
    audio_data : bytes
        ダウンロードするオーディオデータ
    filename : str
        ダウンロード時のファイル名
    text : str
        リンクに表示するテキスト
        
    Returns:
    --------
    str
        HTMLリンク
    """
    b64 = io.BytesIO(audio_data)
    payload = b64.getvalue()
    import base64
    b64_str = base64.b64encode(payload).decode()
    return f'<a href="data:audio/wav;base64,{b64_str}" download="{filename}">{text}</a>'

# 話者IDのオプション
speaker_options = [
    ("日本語標準", "ja"),
    ("英語標準", "en"),
    ("中国語標準", "zh")
]

# ファイルアップロード時の処理
def on_upload_change(change):
    """
    ファイルがアップロードされたときの処理
    """
    if not change.new:
        return
    
    # 処理状況を表示
    status_output.value = "ファイルを処理中..."
    
    try:
        # アップロードされたファイルからテキストを読み込む
        uploaded_file = list(change.new.values())[0]
        text = read_text_file(uploaded_file)
        
        # テキストプレビューを表示
        text_preview.value = text if len(text) <= 1000 else text[:1000] + "..."
        
        # テキストを音声に変換
        speed = float(speed_slider.value)
        language = speaker_dropdown.value
        
        # テキストの前処理
        processed_text = preprocess_text(text, language)
        
        # 音声生成（簡易版を使用）
        audio_path = simple_text_to_speech(processed_text, language, speed)
        
        # 音声を表示
        audio_output.clear_output()
        with audio_output:
            display(Audio(audio_path))
        
        # ダウンロードリンクを作成
        with open(audio_path, 'rb') as f:
            audio_data = f.read()
        
        download_link.value = create_download_link(audio_data, 'output.wav', '音声ファイルをダウンロード')
        
        status_output.value = "処理完了！（注: 現在はデモ音声を生成しています）"
    
    except Exception as e:
        status_output.value = f"エラーが発生しました: {str(e)}"

# 速度変更時の処理
def on_speed_change(change):
    """
    速度スライダーが変更されたときの処理
    """
    speed_value.value = f"発話速度: {change.new}x"

# UIの作成
upload_button = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description='テキストファイルを選択'
)

speaker_dropdown = widgets.Dropdown(
    options=speaker_options,
    value="ja",
    description='言語:',
)

speed_slider = widgets.FloatSlider(
    value=1.0,
    min=0.5,
    max=2.0,
    step=0.1,
    description='速度:',
    continuous_update=False
)

speed_value = widgets.HTML(value="発話速度: 1.0x")
status_output = widgets.HTML(value="ファイルをアップロードしてください。")
text_preview = widgets.Textarea(
    description='テキスト:',
    placeholder='アップロードされたテキストがここに表示されます',
    disabled=True,
    layout=widgets.Layout(width='100%', height='200px')
)

audio_output = widgets.Output()
download_link = widgets.HTML()

# イベントハンドラの登録
upload_button.observe(on_upload_change, names='value')
speed_slider.observe(on_speed_change, names='value')

# ボタンクリック時の処理
def on_convert_button_click(b):
    text = text_input.value
    if not text:
        status_output.value = "テキストが入力されていません。"
        return
    
    status_output.value = "テキストを処理中..."
    
    try:
        # テキストプレビューを表示
        text_preview.value = text if len(text) <= 1000 else text[:1000] + "..."
        
        # テキストを音声に変換
        speed = float(speed_slider.value)
        language = speaker_dropdown.value
        
        # テキストの前処理
        processed_text = preprocess_text(text, language)
        
        # 音声生成（簡易版を使用）
        audio_path = simple_text_to_speech(processed_text, language, speed)
        
        # 音声を表示
        audio_output.clear_output()
        with audio_output:
            display(Audio(audio_path))
        
        # ダウンロードリンクを作成
        with open(audio_path, 'rb') as f:
            audio_data = f.read()
        
        download_link.value = create_download_link(audio_data, 'output.wav', '音声ファイルをダウンロード')
        
        status_output.value = "処理完了！（注: 現在はデモ音声を生成しています）"
    
    except Exception as e:
        status_output.value = f"エラーが発生しました: {str(e)}"

# 直接テキスト入力機能も追加
text_input = widgets.Textarea(
    description='直接入力:',
    placeholder='ここに直接テキストを入力して変換することもできます',
    layout=widgets.Layout(width='100%', height='150px')
)

# テキスト入力による変換ボタン
convert_button = widgets.Button(
    description='テキストを変換',
    button_style='primary',
    tooltip='入力されたテキストを音声に変換します'
)

# イベントハンドラの登録
convert_button.on_click(on_convert_button_click)

# UIの表示
display(widgets.HTML("<h1>Fish-Speech-1.5 日本語テキスト→音声変換アプリ（修正版）</h1>"))
display(widgets.HTML("<p>日本語のテキストファイルをアップロードして、音声に変換します。</p>"))
display(widgets.HTML("<p style='color: orange;'><strong>注意:</strong> 現在はデモ版として動作しており、実際のFish-Speech-1.5音声ではありません。モデルの互換性の問題を修正中です。</p>"))
display(widgets.HBox([upload_button, speaker_dropdown, widgets.VBox([speed_slider, speed_value])]))
display(status_output)
display(text_preview)
display(widgets.HTML("<h3>生成された音声:</h3>"))
display(audio_output)
display(download_link)

# 直接入力UIの表示
display(widgets.HTML("<h3>または直接テキストを入力:</h3>"))
display(text_input)
display(convert_button)

## 高度な使用方法（別アプローチ）

Fish-Speech-1.5の公式デモサイトを利用する方法もあります。以下のURLにアクセスすることで、WebブラウザからFish-Speech-1.5を使用できます。

- [Fish-Speech デモサイト](https://huggingface.co/spaces/fishaudio/fish-speech-1)

上記のデモサイトでは、複数言語（日本語含む）のテキスト入力と音声生成をサポートしています。